In [1]:
from models.nac import NAC
import torch.nn.functional as F
import torch
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from tqdm import tqdm_notebook as tqdm

from data_generator_helper import generate_synthetic_selection_dataset

from models.nac import NeuralAccumulatorCell
import torch
import torchvision.utils as vutils
import numpy as np
import torchvision.models as models
from torchvision import datasets
from tensorboardX import SummaryWriter
import datetime
import os

In [2]:
def reportLoss(loss, epoch):
    print("epoch {},  \t loss {}".format(epoch, loss))
    
def train(model, optimizer, x_train, y_train, epochs, batch_size):
    
    for epoch in range(epochs):
    
        
        for batch in range(len(x_train) // batch_size):
            
            model.train()
            optimizer.zero_grad()

            x_batch_train = x_train[batch:(batch+batch_size),:]
            y_batch_train = y_train[batch:(batch+batch_size),:]

            out = model(x_batch_train)

            loss = F.mse_loss(out, y_batch_train)
            
            if loss != loss:
                break
                print("nan detected")
            
            loss.backward()
            optimizer.step()
        
        if loss != loss:
            break
        
        if epoch % 10 == 0: 
            pass
            #reportLoss(loss.data, epoch)
            
        model.temperature = epoch * 0.005
            
    return test(model,x_train,y_train)
        
def test(model, x_test, y_test):
    
    model.eval()
    output_test = model(x_test)
    loss = F.mse_loss(output_test, y_test)
    
    return loss

In [9]:
init.

<module 'torch.nn.init' from '/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/torch/nn/init.py'>

In [3]:
import math
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F

from torch.nn.parameter import Parameter


class NeuralAccumulatorCell(nn.Module):
    """A Neural Accumulator (NAC) cell [1].
    Attributes:
        in_dim: size of the input sample.
        out_dim: size of the output sample.
    Sources:
        [1]: https://arxiv.org/abs/1808.00508
    """
    def __init__(self, in_dim, out_dim, ini):
        super().__init__()
        self.in_dim = in_dim
        self.out_dim = out_dim
        self.initial = ini
        self.W_hat = Parameter(torch.Tensor(out_dim, in_dim))
        self.M_hat = Parameter(torch.Tensor(out_dim, in_dim))

        self.register_parameter('W_hat', self.W_hat)
        self.register_parameter('M_hat', self.M_hat)
        self.register_parameter('bias', None)

        self._reset_params(ini)

    def _reset_params(self,ini):
        if ini =='Kai_uni':
            init.kaiming_uniform_(self.W_hat)
            init.kaiming_uniform_(self.M_hat)

        if ini =='Xav_norm':
            init.xavier_normal_(self.W_hat)
            init.xavier_normal_(self.M_hat)

        if ini =='Kai_norm':
            init.kaiming_normal_(self.W_hat)
            init.kaiming_normal_(self.M_hat)

        if ini =='Zeros':
            init.zeros_(self.W_hat)
            init.zeros_(self.M_hat)

        if ini =='Ones':
            init.ones_(self.W_hat)
            init.ones_(self.M_hat)

    def forward(self, input):
        W = torch.tanh(self.W_hat) * torch.sigmoid(self.M_hat)
        return F.linear(input, W, self.bias)

    def extra_repr(self):
        return 'in_dim={}, out_dim={}'.format(
            self.in_dim, self.out_dim
        )

In [19]:
import math
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F

from torch.nn.parameter import Parameter
import numpy as np

class NeuralArithmeticLogicUnitCell(nn.Module):
    """A Neural Arithmetic Logic Unit (NALU) cell [1].

    Attributes:
        in_dim: size of the input sample.
        out_dim: size of the output sample.

    Sources:
        [1]: https://arxiv.org/abs/1808.00508
    """
    def __init__(self, in_dim, out_dim, ini):
        super().__init__()
        self.in_dim = in_dim
        self.out_dim = out_dim
        self.eps = 1e-10

        self.G = Parameter(torch.Tensor(out_dim, in_dim))
        self.nac = NeuralAccumulatorCell(in_dim, out_dim, ini)
        self.register_parameter('bias', None)
        self.temperature = 0.005
        
        if ini =='Kai_uni':
            init.kaiming_uniform_(self.G, a=math.sqrt(5))

        if ini =='Xav_norm':
            init.xavier_normal_(self.G)

        if ini =='Kai_norm':
            init.kaiming_normal_(self.G)

        if ini =='Zeros':
            init.zeros_(self.G)

        if ini =='Ones':
            init.ones_(self.G)


    def forward(self, input):
        a = self.nac(input)

        g = torch.sigmoid(F.linear(input, self.G, self.bias) * self.temperature)
        add_sub = g * a
        log_input = torch.log(torch.abs(input) + self.eps)
        m = torch.exp(self.nac(log_input))
        mul_div = (1 - g) * m
        y = add_sub + mul_div
        return y
    
    def extra_repr(self):
        return 'in_dim={}, out_dim={}'.format(
            self.in_dim, self.out_dim
        )


In [21]:
test_per_range = 10
sample_size = 100
set_size = 100

in_dim = sample_size
hidden_dim = 1
out_dim = 2
num_layers = 1

lr = 0.01
epochs = 1000
batch_size = 1

#values = np.linspace(1,1000,10)
inits = ['Kai_uni','Xav_norm','Kai_norm','Zeros', 'Ones']
train_loss = np.zeros((len(inits),10))
test_loss = np.zeros((len(inits),10))

for j, ini in tqdm(enumerate(inits)):
    
    print(ini)
    
    min_value = 0
    max_value = 1
    
    train_acc = 0
    test_acc = 0
    for i in range(test_per_range):

        model = NeuralArithmeticLogicUnitCell(in_dim, out_dim, ini)
        optimizer = torch.optim.RMSprop(model.parameters(),lr=lr)

        x_train, y_train, boundaries = generate_synthetic_selection_dataset(min_value, max_value,
                                                                    sample_size, set_size, boundaries = None)

        x_test, y_test, _ = generate_synthetic_selection_dataset(min_value, max_value,
                                                                    sample_size, set_size, boundaries = boundaries)

        train_loss[j, i] = train(model, optimizer, x_train, y_train, epochs, batch_size)

        test_loss[j, i]  = test(model, x_test, y_test)
    
        

Zeros


AttributeError: module 'torch.nn.init' has no attribute 'zeros_'